In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

pd.set_option("mode.copy_on_write", True)

repo_root = Path("../..")
int_dir = repo_root / "20_intermediate_file"
# source file
overdose = int_dir / "overdose_03-15.tsv"
pop = int_dir / "population_00-19.csv"

# output file
out_dir = int_dir
out_file = int_dir / "overdose_211_counties.parquet"

overdose = pd.read_csv(overdose, sep="\t")
pop = pd.read_csv(pop)

In [2]:
pop = pop[["STNAME", "FIPS", "Year", "Population"]]
pop

,STNAME,FIPS,Year,Population
0,Alabama,1001,2000,44021
1,Alabama,1003,2000,141342
2,Alabama,1005,2000,29015
3,Alabama,1007,2000,19913
4,Alabama,1009,2000,51107
...,...,...,...,...
62265,Wyoming,56037,2019,42917
62266,Wyoming,56039,2019,23385
62267,Wyoming,56041,2019,20196
62268,Wyoming,56043,2019,7824


In [3]:
merged = overdose.merge(pop, how="left", on=["FIPS", "Year"], validate="1:1")
merged

,FIPS,County,Year,Deaths,STNAME,Population
0,1003,"Baldwin County, AL",2003,10,Alabama,151509
1,1003,"Baldwin County, AL",2004,18,Alabama,156266
2,1003,"Baldwin County, AL",2005,14,Alabama,162183
3,1003,"Baldwin County, AL",2006,11,Alabama,168121
4,1003,"Baldwin County, AL",2007,24,Alabama,172404
...,...,...,...,...,...,...
7850,56025,"Natrona County, WY",2012,12,Wyoming,78620
7851,56025,"Natrona County, WY",2013,14,Wyoming,81175
7852,56025,"Natrona County, WY",2014,17,Wyoming,81453
7853,56025,"Natrona County, WY",2015,13,Wyoming,82218


In [4]:
no_missing = merged.groupby("FIPS", as_index=False).filter(lambda g: len(g) == 13)
no_missing

,FIPS,County,Year,Deaths,STNAME,Population
0,1003,"Baldwin County, AL",2003,10,Alabama,151509
1,1003,"Baldwin County, AL",2004,18,Alabama,156266
2,1003,"Baldwin County, AL",2005,14,Alabama,162183
3,1003,"Baldwin County, AL",2006,11,Alabama,168121
4,1003,"Baldwin County, AL",2007,24,Alabama,172404
...,...,...,...,...,...,...
7824,55133,"Waukesha County, WI",2011,42,Wisconsin,390895
7825,55133,"Waukesha County, WI",2012,41,Wisconsin,392893
7826,55133,"Waukesha County, WI",2013,39,Wisconsin,394257
7827,55133,"Waukesha County, WI",2014,38,Wisconsin,395518


In [5]:
has_missing = merged.groupby("FIPS", as_index=False).filter(lambda g: len(g) < 13)
has_missing.groupby("County", as_index=False).max().sort_values("Population")[-20:]

,County,FIPS,Year,Deaths,STNAME,Population
497,"Orange County, NY",36071,2015,67,New York,375911
165,"Dakota County, MN",27037,2015,39,Minnesota,414245
81,"Cameron County, TX",48061,2015,19,Texas,419062
539,"Prince William County, VA",51153,2015,29,Virginia,451242
493,"Onondaga County, NY",36067,2015,89,New York,468302
352,"Lake County, IN",18089,2015,59,Indiana,495972
465,"Morris County, NJ",34027,2015,49,New Jersey,496050
508,"Passaic County, NJ",34031,2015,63,New Jersey,504556
332,"Kane County, IL",17089,2015,43,Illinois,528562
185,"Douglas County, NE",31055,2015,52,Nebraska,549313


> NY and NJ have many missing, so we opt to drop them for final analysis.

In [6]:
has_missing[(has_missing.STNAME == "Texas") & (has_missing.Population > 400_000)]

,FIPS,County,Year,Deaths,STNAME,Population
6669,48061,"Cameron County, TX",2009,18,Texas,400303
6670,48061,"Cameron County, TX",2010,13,Texas,407608
6671,48061,"Cameron County, TX",2012,15,Texas,415729
6672,48061,"Cameron County, TX",2015,17,Texas,419062
6746,48157,"Fort Bend County, TX",2003,11,Texas,418747
6747,48157,"Fort Bend County, TX",2005,13,Texas,462953
6748,48157,"Fort Bend County, TX",2006,18,Texas,490916
6749,48157,"Fort Bend County, TX",2007,19,Texas,516564
6750,48157,"Fort Bend County, TX",2008,13,Texas,542957
6751,48157,"Fort Bend County, TX",2009,25,Texas,569130


> Observe that in the earlier years, Fort Bend County and Hidalgo County have a few missing entries, and they both only reported `Drug poisonings (overdose) Unintentional (X40-X44)` overdose category. Considering their trends and high populations, it is likely that the actual overdose deaths fell just under 10, so we opt to impute 8.

In [7]:
impute = overdose.loc[[6747, 6812, 6813]]
impute.Year -= 1
impute.Deaths = 8
impute

,FIPS,County,Year,Deaths
6747,48157,"Fort Bend County, TX",2004,8
6812,48215,"Hidalgo County, TX",2003,8
6813,48215,"Hidalgo County, TX",2005,8


In [8]:
imputed_overdose = pd.concat([overdose, impute], ignore_index=True)
merged = imputed_overdose.merge(pop, how="left", on=["FIPS", "Year"], validate="1:1")
merged

,FIPS,County,Year,Deaths,STNAME,Population
0,1003,"Baldwin County, AL",2003,10,Alabama,151509
1,1003,"Baldwin County, AL",2004,18,Alabama,156266
2,1003,"Baldwin County, AL",2005,14,Alabama,162183
3,1003,"Baldwin County, AL",2006,11,Alabama,168121
4,1003,"Baldwin County, AL",2007,24,Alabama,172404
...,...,...,...,...,...,...
7853,56025,"Natrona County, WY",2015,13,Wyoming,82218
7854,56037,"Sweetwater County, WY",2014,13,Wyoming,44996
7855,48157,"Fort Bend County, TX",2004,8,Texas,441139
7856,48215,"Hidalgo County, TX",2003,8,Texas,632475


# Threshold per analysis

In [9]:
treated = ["Texas", "Florida", "Washington"]
exclude = {"New Jersey", "New York", *treated}
# exclude = {"New Jersey", "New York"}
change_years = {"Texas": 2007, "Florida": 2010, "Washington": 2012}

state = "Texas"
p_m = 4
excluded = merged[~merged.STNAME.isin(exclude - {state})]
excluded

,FIPS,County,Year,Deaths,STNAME,Population
0,1003,"Baldwin County, AL",2003,10,Alabama,151509
1,1003,"Baldwin County, AL",2004,18,Alabama,156266
2,1003,"Baldwin County, AL",2005,14,Alabama,162183
3,1003,"Baldwin County, AL",2006,11,Alabama,168121
4,1003,"Baldwin County, AL",2007,24,Alabama,172404
...,...,...,...,...,...,...
7853,56025,"Natrona County, WY",2015,13,Wyoming,82218
7854,56037,"Sweetwater County, WY",2014,13,Wyoming,44996
7855,48157,"Fort Bend County, TX",2004,8,Texas,441139
7856,48215,"Hidalgo County, TX",2003,8,Texas,632475


In [10]:
state = "Texas"
change_year = change_years[state]
state_df = merged[merged.Year.between(change_year - p_m, change_year + p_m, "left")]
state_df.Year.min(), state_df.Year.max()

(2003, 2010)

In [11]:
num_years = state_df.Year.nunique()
has_missing = merged.groupby("FIPS", as_index=False).filter(
    lambda g: len(g) < num_years
)
thresh = has_missing.Population.max()
print(f"Population threshold: {thresh}")

Population threshold: 374176


In [12]:
state_df = state_df[state_df.Population > thresh]
print(f"Number of counties: {state_df.FIPS.nunique()}")

Number of counties: 177


In [13]:
state = "Washington"
change_year = change_years[state]
state_df = merged[~merged.STNAME.isin(exclude - {state})]
state_df = state_df[state_df.Year.between(change_year - p_m, change_year + p_m, "left")]
print(state_df.Year.min(), state_df.Year.max())

num_years = state_df.Year.nunique()
thresh = (
    state_df.groupby("FIPS", as_index=False)
    .Population.filter(lambda g: len(g) < num_years)
    .max()
)
print(f"Population threshold: {thresh}")

filtered = state_df.groupby("FIPS", as_index=False).filter(
    lambda g: g.Population.max() > thresh
)
print(f"Number of counties: {filtered.FIPS.nunique()}")

2008 2015
Population threshold: 374176
Number of counties: 131


In [14]:
def analysis_scope(overdose_pop, state, p_m=4):
    print("====", state, "====")
    change_year = change_years[state]
    state_df = overdose_pop[~overdose_pop.STNAME.isin(exclude - {state})]
    state_df = state_df[
        state_df.Year.between(change_year - p_m, change_year + p_m, "left")
    ]
    print(f"Year range: {state_df.Year.min()} to {state_df.Year.max()}")

    num_years = state_df.Year.nunique()
    thresh = (
        state_df.groupby("FIPS", as_index=False)
        .Population.filter(lambda g: len(g) < num_years)
        .max()
    )
    print(f"Population threshold: {thresh}")

    filtered = state_df.groupby("FIPS", as_index=False).filter(
        lambda g: g.Population.max() > thresh
    )
    print(f"Number of counties: {filtered.FIPS.nunique()}\n")
    filtered.to_parquet(out_dir / f"overdose_analysis_scope_{state}.parquet")

In [15]:
analysis_scope(merged, "Texas")
analysis_scope(merged, "Florida")
analysis_scope(merged, "Washington")

==== Texas ====
Year range: 2003 to 2010
Population threshold: 518599
Number of counties: 93

==== Florida ====
Year range: 2006 to 2013
Population threshold: 437900
Number of counties: 117

==== Washington ====
Year range: 2008 to 2015
Population threshold: 374176
Number of counties: 131



# Population threshold
We want to select counties that have no missing overdose data during this time period. To do that, we can find a population threshold, which is the max population in the `has_missing` group.

We want to work on the same counties every year in our later analysis, so we filter based on a county's max population during this time. 

Note if we require max pop to be larger than this threshold, we exclude all counties that have missing data.

In [16]:
pop_thresh = has_missing["Population"].max()
max_pops = no_missing.groupby("County")["Population"].max()
counties = max_pops[max_pops > pop_thresh]
len(counties)

161

In [17]:
big_counties = no_missing[no_missing["County"].isin(counties.index)]
big_counties

,FIPS,County,Year,Deaths,STNAME,Population
56,1073,"Jefferson County, AL",2003,37,Alabama,657513
57,1073,"Jefferson County, AL",2004,42,Alabama,656023
58,1073,"Jefferson County, AL",2005,41,Alabama,654919
59,1073,"Jefferson County, AL",2006,55,Alabama,655893
60,1073,"Jefferson County, AL",2007,67,Alabama,655163
...,...,...,...,...,...,...
7824,55133,"Waukesha County, WI",2011,42,Wisconsin,390895
7825,55133,"Waukesha County, WI",2012,41,Wisconsin,392893
7826,55133,"Waukesha County, WI",2013,39,Wisconsin,394257
7827,55133,"Waukesha County, WI",2014,38,Wisconsin,395518


## State-wise threshold

The above approach is ok, but we are left with only 46 counties... what if we try state-wise?

Note this may still have a slight bias towards dropping counties with less opioid problems, since the thresholds very precisely target the largest such counties

In [18]:
state_thresholds = has_missing.groupby("STNAME")["Population"].max()
state_thresholds, state_thresholds.shape

(STNAME
 Alabama           229498
 Arizona           139100
 Arkansas          224600
 California         94466
 Colorado           14048
 Connecticut       152330
 Florida           172607
 Georgia           143991
 Hawaii            163993
 Idaho             202542
 Illinois          147699
 Indiana           155906
 Iowa              142874
 Kansas            118265
 Kentucky          124224
 Louisiana         132337
 Maine             108284
 Maryland          155775
 Massachusetts      71399
 Michigan          163047
 Minnesota         156068
 Mississippi       248989
 Missouri          117801
 Montana            95940
 Nebraska          162597
 Nevada             55995
 New Hampshire      90041
 New Jersey        128449
 New Mexico         70666
 New York          234488
 North Carolina    222051
 North Dakota      166792
 Ohio              176185
 Oklahoma          133241
 Oregon            169631
 Pennsylvania      160641
 Rhode Island       85509
 South Carolina    175318
 Sou

In [19]:
# add 'District of Columbia'
state_thresholds["District of Columbia"] = 0

In [20]:
max_pops = no_missing.groupby("County", as_index=False).max()
thresholds = state_thresholds.loc[max_pops["STNAME"]].values
fips = max_pops["FIPS"][max_pops["Population"] > thresholds]
fips

KeyError: "['Delaware'] not in index"

In [ ]:
big_counties = no_missing[no_missing["FIPS"].isin(fips)]
# big_counties.to_parquet(out_file, compression="zstd")
big_counties

,FIPS,County,Year,Deaths,STNAME,Population
56,1073,"Jefferson County, AL",2003,37,Alabama,657513
57,1073,"Jefferson County, AL",2004,42,Alabama,656023
58,1073,"Jefferson County, AL",2005,41,Alabama,654919
59,1073,"Jefferson County, AL",2006,55,Alabama,655893
60,1073,"Jefferson County, AL",2007,67,Alabama,655163
...,...,...,...,...,...,...
7824,55133,"Waukesha County, WI",2011,42,Wisconsin,390895
7825,55133,"Waukesha County, WI",2012,41,Wisconsin,392893
7826,55133,"Waukesha County, WI",2013,39,Wisconsin,394257
7827,55133,"Waukesha County, WI",2014,38,Wisconsin,395518
